In [1]:
"""
ECG Analysis Pipeline: From PDF to Diagnosis
-------------------------------------------
This module provides functions to:
1. Digitize ECG from PDF files
2. Process the digitized ECG signal
3. Make diagnoses using a trained model
"""

import cv2
import numpy as np
import os
import tensorflow as tf
import pickle
from scipy.interpolate import interp1d
from collections import Counter
from pdf2image import convert_from_path
import matplotlib.pyplot as plt  # Added for visualization

def digitize_ecg_from_pdf(pdf_path, output_file='calibrated_ecg.dat', debug=False):
    """
    Process an ECG PDF file and convert it to a .dat signal file.
    
    Args:
        pdf_path (str): Path to the ECG PDF file
        output_file (str): Path to save the output .dat file (default: 'calibrated_ecg.dat')
        debug (bool): Whether to print debug information
    
    Returns:
        str: Path to the created .dat file
    """
    if debug:
        print(f"Starting ECG digitization from PDF: {pdf_path}")
    
    # Convert PDF to image
    images = convert_from_path(pdf_path)
    temp_image_path = 'temp_ecg_image.jpg'
    images[0].save(temp_image_path, 'JPEG')
    
    if debug:
        print(f"Converted PDF to image: {temp_image_path}")
    
    # Load the image
    img = cv2.imread(temp_image_path, cv2.IMREAD_GRAYSCALE)
    height, width = img.shape
    
    if debug:
        print(f"Image dimensions: {width}x{height}")
    
    # Fixed calibration parameters
    calibration = {
        'seconds_per_pixel': 2.0 / 197.0,  # 197 pixels = 2 seconds
        'mv_per_pixel': 1.0 / 78.8,        # 78.8 pixels = 1 mV
    }
    
    if debug:
        print(f"Calibration parameters: {calibration}")
    
    # Calculate layer boundaries using percentages
    layer1_start = int(height * 35.35 / 100)
    layer1_end = int(height * 51.76 / 100)
    layer2_start = int(height * 51.82 / 100)
    layer2_end = int(height * 69.41 / 100)
    layer3_start = int(height * 69.47 / 100)
    layer3_end = int(height * 87.06 / 100)
    
    if debug:
        print(f"Layer 1 boundaries: {layer1_start}-{layer1_end}")
        print(f"Layer 2 boundaries: {layer2_start}-{layer2_end}")
        print(f"Layer 3 boundaries: {layer3_start}-{layer3_end}")
    
    # Crop each layer
    layers = [
        img[layer1_start:layer1_end, :],  # Layer 1
        img[layer2_start:layer2_end, :],  # Layer 2
        img[layer3_start:layer3_end, :]   # Layer 3
    ]
    
    # Process each layer to extract waveform contours
    signals = []
    time_points = []
    layer_duration = 10.0  # Each layer is 10 seconds long
    
    for i, layer in enumerate(layers):
        if debug:
            print(f"Processing layer {i+1}...")
        
        # Binary thresholding
        _, binary = cv2.threshold(layer, 200, 255, cv2.THRESH_BINARY_INV)
        
        # Detect contours
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        waveform_contour = max(contours, key=cv2.contourArea)  # Largest contour is the ECG
        
        if debug:
            print(f"  - Found {len(contours)} contours")
            print(f"  - Selected contour with {len(waveform_contour)} points")
        
        # Sort contour points and extract coordinates
        sorted_contour = sorted(waveform_contour, key=lambda p: p[0][0])
        x_coords = np.array([point[0][0] for point in sorted_contour])
        y_coords = np.array([point[0][1] for point in sorted_contour])
        
        # Calculate isoelectric line (one-third from the bottom)
        isoelectric_line_y = layer.shape[0] * 0.6
        
        # Convert to time using fixed layer duration
        x_min, x_max = np.min(x_coords), np.max(x_coords)
        time = (x_coords - x_min) / (x_max - x_min) * layer_duration
        
        # Calculate signal in millivolts and apply baseline correction
        signal_mv = (isoelectric_line_y - y_coords) * calibration['mv_per_pixel']
        signal_mv = signal_mv - np.mean(signal_mv)
        
        if debug:
            print(f"  - Layer {i+1} signal range: {np.min(signal_mv):.2f} mV to {np.max(signal_mv):.2f} mV")
        
        # Store the time points and calibrated signal
        time_points.append(time)
        signals.append(signal_mv)
    
    # Combine signals with proper time alignment
    total_duration = layer_duration * len(layers)
    sampling_frequency = 500  # Standard ECG frequency
    num_samples = int(total_duration * sampling_frequency)
    combined_time = np.linspace(0, total_duration, num_samples)
    combined_signal = np.zeros(num_samples)
    
    if debug:
        print(f"Combining signals with {sampling_frequency} Hz sampling rate, total duration: {total_duration}s")
    
    # Place each lead at the correct time position
    for i, (time, signal) in enumerate(zip(time_points, signals)):
        start_time = i * layer_duration
        mask = (combined_time >= start_time) & (combined_time < start_time + layer_duration)
        relevant_times = combined_time[mask]
        interpolated_signal = np.interp(relevant_times, start_time + time, signal)
        combined_signal[mask] = interpolated_signal
        
        if debug:
            print(f"  - Added layer {i+1} signal from {start_time}s to {start_time + layer_duration}s")
    
    # Baseline correction and amplitude scaling
    combined_signal = combined_signal - np.mean(combined_signal)
    signal_peak = np.max(np.abs(combined_signal))
    target_amplitude = 2.0  # Target peak amplitude in mV
    
    if debug:
        print(f"Signal peak before scaling: {signal_peak:.2f} mV")
    
    if signal_peak > 0 and (signal_peak < 0.5 or signal_peak > 4.0):
        scaling_factor = target_amplitude / signal_peak
        combined_signal = combined_signal * scaling_factor
        if debug:
            print(f"Applied scaling factor: {scaling_factor:.2f}")
            print(f"Signal peak after scaling: {np.max(np.abs(combined_signal)):.2f} mV")
    
    # Convert to 16-bit integers and save as .dat file
    adc_gain = 1000.0  # Standard gain: 1000 units per mV
    int_signal = (combined_signal * adc_gain).astype(np.int16)
    int_signal.tofile(output_file)
    
    if debug:
        print(f"Saved signal to {output_file} with {len(int_signal)} samples")
        print(f"Integer signal range: {np.min(int_signal)} to {np.max(int_signal)}")
    
    # Clean up temporary files
    if os.path.exists(temp_image_path):
        os.remove(temp_image_path)
        if debug:
            print(f"Removed temporary image: {temp_image_path}")
    
    return output_file

def visualize_ecg_signal(signal, sampling_rate=500, title="Digitized ECG Signal"):
    """
    Visualize an ECG signal with proper time axis.
    
    Parameters:
    -----------
    signal : numpy.ndarray
        ECG signal data
    sampling_rate : int
        Sampling rate in Hz
    title : str
        Plot title
    """
    # Calculate time axis
    time = np.arange(len(signal)) / sampling_rate
    
    # Create figure with appropriate size
    plt.figure(figsize=(15, 5))
    plt.plot(time, signal)
    plt.title(title)
    plt.xlabel('Time (seconds)')
    plt.ylabel('Amplitude (mV)')
    plt.grid(True)
    
    # Add 1mV scale bar
    plt.plot([1, 1], [-0.5, 0.5], 'r-', linewidth=2)
    plt.text(1.1, 0, '1mV', va='center')
    
    # Add time scale bar (1 second)
    y_min = np.min(signal)
    plt.plot([1, 2], [y_min, y_min], 'r-', linewidth=2)
    plt.text(1.5, y_min - 0.1, '1s', ha='center')
    
    plt.tight_layout()
    plt.show()

def read_lead_i_long_dat_file(dat_file_path, sampling_rate=500, data_format='16', scale_factor=0.001):
    """
    Read a 30-second pure Lead I .dat file directly and properly scale it
    
    Parameters:
    -----------
    dat_file_path : str
        Path to the .dat file (with or without .dat extension)
    sampling_rate : int
        Sampling rate in Hz (default 500Hz)
    data_format : str
        Data format of the binary file: '16' for 16-bit integers, '32' for 32-bit floats
    scale_factor : float
        Scale factor to convert units (0.001 for converting µV to mV)
        
    Returns:
    --------
    numpy.ndarray
        ECG signal data for Lead I with shape (total_samples,)
    """
    # Ensure the path ends with .dat
    if not dat_file_path.endswith('.dat'):
        dat_file_path += '.dat'
    
    # Expected samples for full 30 seconds
    expected_samples = sampling_rate * 30
    
    # Read the binary data
    try:
        if data_format == '16':
            # 16-bit signed integers (common format for ECG)
            data = np.fromfile(dat_file_path, dtype=np.int16)
        elif data_format == '32':
            # 32-bit floating point (less common)
            data = np.fromfile(dat_file_path, dtype=np.float32)
        else:
            raise ValueError(f"Unsupported data format: {data_format}")
    
        # Apply scaling to convert µV to mV
        signal = data * scale_factor
        
        # Handle if signal is not exactly 30 seconds
        if len(signal) < expected_samples:
            # Pad with zeros if too short
            padded_signal = np.zeros(expected_samples)
            padded_signal[:len(signal)] = signal
            signal = padded_signal
        elif len(signal) > expected_samples:
            # Truncate if too long
            signal = signal[:expected_samples]
        
        return signal
        
    except Exception as e:
        raise

def segment_signal(signal, sampling_rate=500):
    """
    Segment a 30-second signal into three 10-second segments
    
    Parameters:
    -----------
    signal : numpy.ndarray
        The full signal to segment
    sampling_rate : int
        Sampling rate in Hz
        
    Returns:
    --------
    list
        List of three 10-second signal segments
    """
    # Calculate samples per segment (10 seconds)
    segment_samples = sampling_rate * 10
    
    # Expected samples for full 30 seconds
    expected_samples = sampling_rate * 30
    
    # Ensure the signal is 30 seconds long
    if len(signal) != expected_samples:
        # Resample to 30 seconds
        x = np.linspace(0, 1, len(signal))
        x_new = np.linspace(0, 1, expected_samples)
        f = interp1d(x, signal, kind='linear', bounds_error=False, fill_value="extrapolate")
        signal = f(x_new)
    
    # Split the signal into three 10-second segments
    segments = []
    for i in range(3):
        start_idx = i * segment_samples
        end_idx = (i + 1) * segment_samples
        segment = signal[start_idx:end_idx]
        segments.append(segment)
        
    return segments

def process_segment(segment, sampling_rate=500):
    """
    Process a segment of ECG data to ensure it's properly formatted for the model
    
    Parameters:
    -----------
    segment : numpy.ndarray
        Raw ECG segment
    sampling_rate : int
        Sampling rate of the ECG
        
    Returns:
    --------
    numpy.ndarray
        Processed segment ready for model input
    """
    # Ensure correct length (5000 samples for 10 seconds)
    if len(segment) != 5000:
        x = np.linspace(0, 1, len(segment))
        x_new = np.linspace(0, 1, 5000)
        f = interp1d(x, segment, kind='linear', bounds_error=False, fill_value="extrapolate")
        segment = f(x_new)
    
    return segment

def predict_with_voting(dat_file_path, model_path, mlb_path=None, sampling_rate=500, scale_factor=0.001, debug=False):
    """
    Process a 30-second .dat file, properly scale it, split it into three 10-second segments,
    make predictions on each segment, and return the class with highest average probability.
    
    Parameters:
    -----------
    dat_file_path : str
        Path to the .dat file
    model_path : str
        Path to the saved model (.h5 file)
    mlb_path : str, optional
        Path to the saved MultiLabelBinarizer pickle file for label decoding
    sampling_rate : int
        Sampling rate in Hz (default 500Hz)
    scale_factor : float
        Scale factor to convert units (0.001 for converting µV to mV)
    debug : bool
        Whether to print debug information
        
    Returns:
    --------
    dict
        Dictionary containing segment predictions and final class probabilities
    """
    try:
        # Step 1: Read the 30-second ECG data (pure Lead I) and apply scaling
        if debug:
            print(f"Reading signal from {dat_file_path}")
        
        full_signal = read_lead_i_long_dat_file(
            dat_file_path, 
            sampling_rate=sampling_rate,
            scale_factor=scale_factor
        )
        
        if debug:
            print(f"Signal loaded: {len(full_signal)} samples, range: {np.min(full_signal):.2f} to {np.max(full_signal):.2f} mV")
        
        # Step 2: Split into three 10-second segments
        segments = segment_signal(full_signal, sampling_rate)
        
        if debug:
            print(f"Split into {len(segments)} segments of {len(segments[0])} samples each")
        
        # Step 3: Load the model (load once to improve performance)
        if debug:
            print(f"Loading model from {model_path}")
        
        model = tf.keras.models.load_model(model_path)
        
        # Load MLB if provided
        mlb = None
        if mlb_path and os.path.exists(mlb_path):
            if debug:
                print(f"Loading label binarizer from {mlb_path}")
            with open(mlb_path, 'rb') as f:
                mlb = pickle.load(f)
        
        # Step 4: Process each segment and collect predictions
        segment_results = []
        all_predictions = []
        
        for i, segment in enumerate(segments):
            if debug:
                print(f"Processing segment {i+1}...")
            
            # Process the segment to ensure it's properly formatted
            processed_segment = process_segment(segment)
            
            # Reshape for model input (batch, time, channels)
            X = processed_segment.reshape(1, 5000, 1)
            
            # Make predictions
            predictions = model.predict(X, verbose=0)
            all_predictions.append(predictions[0])
            
            # Process segment results
            segment_result = {"raw_predictions": predictions[0].tolist()}
            
            # Decode labels if MLB is provided
            if mlb is not None:
                # Add class probabilities
                class_probs = {}
                for j, class_name in enumerate(mlb.classes_):
                    class_probs[class_name] = float(predictions[0][j])
                
                segment_result["class_probabilities"] = class_probs
            
            segment_results.append(segment_result)
        
        # Step 5: Calculate average probabilities across all segments
        final_result = {"segment_results": segment_results}
        
        # Average the raw predictions
        avg_predictions = np.mean(all_predictions, axis=0)
        final_result["averaged_raw_predictions"] = avg_predictions.tolist()
        
        # Calculate final class probabilities (average across segments)
        if mlb is not None:
            # Calculate average probability for each class
            final_class_probs = {}
            for cls_idx, cls_name in enumerate(mlb.classes_):
                final_class_probs[cls_name] = float(np.mean([pred[cls_idx] for pred in all_predictions]))
            
            # Find the class with highest average probability
            top_class = max(final_class_probs.items(), key=lambda x: x[1])
            top_class_name = top_class[0]
            
            final_result["final_class_probabilities"] = final_class_probs
            final_result["top_class"] = top_class_name
            
            if debug:
                print(f"Top class by average probability: {top_class_name} ({top_class[1]:.2f})")
        
        return final_result
        
    except Exception as e:
        if debug:
            print(f"Error in predict_with_voting: {str(e)}")
        return {"error": str(e)}

def analyze_ecg_pdf(pdf_path, model_path, mlb_path=None, temp_dat_file='calibrated_ecg.dat', cleanup=True, debug=False, visualize=False):
    """
    Complete ECG analysis pipeline: digitizes a PDF ECG, analyzes it with the model,
    and returns the diagnosis with highest probability.
    
    Args:
        pdf_path (str): Path to the ECG PDF file
        model_path (str): Path to the model (.h5) file
        mlb_path (str, optional): Path to the MultiLabelBinarizer file
        temp_dat_file (str, optional): Path to save the temporary digitized file
        cleanup (bool, optional): Whether to remove temporary files after processing
        debug (bool, optional): Whether to print debug information
        visualize (bool, optional): Whether to visualize the digitized signal
    
    Returns:
        dict: {
            "diagnosis": str,  # Top diagnosis (highest average probability)
            "probability": float,  # Probability of top diagnosis
            "all_probabilities": dict,  # All diagnoses with probabilities
            "digitized_file": str  # Path to digitized file (if cleanup=False)
        }
    """
    # Silence TensorFlow warnings
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    
    if debug:
        print(f"Starting ECG analysis pipeline for {pdf_path}")
    
    # 1. Digitize ECG from PDF to DAT file
    dat_file_path = digitize_ecg_from_pdf(pdf_path, output_file=temp_dat_file, debug=debug)
    
    # Visualize the digitized signal if requested
    if visualize:
        signal = read_lead_i_long_dat_file(dat_file_path, scale_factor=0.001)
        visualize_ecg_signal(signal, title=f"Digitized ECG from {os.path.basename(pdf_path)}")
    
    # 2. Process DAT file with model
    if debug:
        print("Processing digitized signal with model...")
    
    results = predict_with_voting(
        dat_file_path,
        model_path,
        mlb_path,
        scale_factor=0.001,  # Convert microvolts to millivolts
        debug=debug
    )
    
    # 3. Extract top diagnosis (highest probability)
    top_diagnosis = {
        "diagnosis": None,
        "probability": 0.0,
        "all_probabilities": {},
        "digitized_file": dat_file_path
    }
    
    # If we have class probabilities, find the highest one
    if "final_class_probabilities" in results:
        probs = results["final_class_probabilities"]
        top_diagnosis["all_probabilities"] = probs
        
        # Use the top class directly from the results
        if "top_class" in results:
            top_diagnosis["diagnosis"] = results["top_class"]
            top_diagnosis["probability"] = probs[results["top_class"]]
            
    # Clean up temporary files if requested
    if cleanup and os.path.exists(dat_file_path):
        if debug:
            print(f"Cleaning up temporary file: {dat_file_path}")
        os.remove(dat_file_path)
        top_diagnosis.pop("digitized_file")
    
    if debug:
        print(f"Analysis complete. Diagnosis: {top_diagnosis['diagnosis']} (Probability: {top_diagnosis['probability']:.2f})")
    
    return top_diagnosis

# Example usage
if __name__ == "__main__":
    # Path configuration
    sample_pdf = 'sample.pdf'
    model_path = 'deep-multiclass.h5'  # Update with actual path
    mlb_path = 'deep-multiclass.pkl'     # Update with actual path
    
    # Analyze ECG with debug output and visualization
    result = analyze_ecg_pdf(
        sample_pdf, 
        model_path, 
        mlb_path, 
        cleanup=False,  # Keep the digitized file
        debug=False,     # Print debug information
        visualize=False  # Visualize the digitized signal
    )
    
    # Display result
    if result["diagnosis"]:
        print(f"Diagnosis: {result['diagnosis']} ")

    else:
        print("No clear diagnosis found")

c:\Users\ranaa\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Diagnosis: NORM 
